In [1]:
import numpy as np
import pandas as pd

In [2]:
path = '/opt/ml/input/data/train/'

train = pd.read_csv(path + 'train_ratings.csv')

In [3]:
# 일단 본거는 전부 Positive feedback (1값)으로 처리, 시간 제거
train['rating'] = 1.0 # implicit feedback
train.drop(['time'],axis=1,inplace=True)

In [4]:
train.head()

,user,item,rating
0,11,4643,1.0
1,11,170,1.0
2,11,531,1.0
3,11,616,1.0
4,11,2140,1.0


In [5]:
train.nunique()

user      31360
item       6807
rating        1
dtype: int64

In [6]:
df = pd.pivot_table(train, index='user',columns='item', values='rating')

In [7]:
df = df.fillna(0)

In [8]:
df

item,1,2,3,4,5,6,7,8,9,10,...,116823,117176,117533,117881,118696,118700,118900,118997,119141,119145
user,,,,,,,,,,,,,,,,,,,,,
11,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
18,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138473,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
138475,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
138486,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
# class EASE:
#     def __init__(self, _lambda):
#         self.B = None
#         self._lambda = _lambda

#     def train(self, X):
#         G = X.T @ X  # G = X'X
#         diag_indices = list(range(G.shape[0]))
#         G[diag_indices, diag_indices] += self._lambda  # X'X + λI
#         P = np.linalg.inv(G)  # P = (X'X + λI)^(-1)
        
#         self.B = P / -np.diag(P)  # - P_{ij} / P_{jj} if i ≠ j
#         min_dim = min(*self.B.shape)  
#         self.B[range(min_dim), range(min_dim)] = 0  # 대각행렬 원소만 0으로 만들어주기 위해

#     def forward(self, user_row):
#         return user_row @ self.B

In [10]:
X = df  # 학습 데이터

In [11]:
_lambda = 300

In [12]:
G = X.T @ X

In [13]:
G

item,1,2,3,4,5,6,7,8,9,10,...,116823,117176,117533,117881,118696,118700,118900,118997,119141,119145
item,,,,,,,,,,,,,,,,,,,,,
1,12217.0,2282.0,470.0,22.0,383.0,2564.0,473.0,32.0,38.0,2571.0,...,117.0,81.0,20.0,14.0,114.0,28.0,27.0,29.0,58.0,38.0
2,2282.0,3364.0,236.0,10.0,227.0,744.0,227.0,21.0,22.0,1028.0,...,44.0,26.0,3.0,4.0,53.0,13.0,10.0,15.0,27.0,13.0
3,470.0,236.0,734.0,7.0,115.0,242.0,107.0,8.0,9.0,255.0,...,6.0,1.0,1.0,0.0,7.0,1.0,0.0,0.0,1.0,0.0
4,22.0,10.0,7.0,43.0,9.0,13.0,7.0,0.0,0.0,8.0,...,3.0,1.0,0.0,0.0,3.0,1.0,1.0,1.0,0.0,0.0
5,383.0,227.0,115.0,9.0,590.0,126.0,112.0,8.0,5.0,173.0,...,6.0,3.0,1.0,2.0,8.0,0.0,0.0,5.0,3.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118700,28.0,13.0,1.0,1.0,0.0,18.0,1.0,0.0,0.0,9.0,...,10.0,21.0,5.0,11.0,11.0,54.0,16.0,8.0,8.0,4.0
118900,27.0,10.0,0.0,1.0,0.0,17.0,1.0,1.0,1.0,7.0,...,12.0,20.0,9.0,9.0,10.0,16.0,60.0,11.0,9.0,6.0
118997,29.0,15.0,0.0,1.0,5.0,12.0,5.0,1.0,0.0,5.0,...,15.0,14.0,1.0,6.0,19.0,8.0,11.0,52.0,7.0,5.0


In [14]:
diag_indices = list(range(G.shape[0]))
diag_indices

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [15]:
len(diag_indices)

6807

In [16]:
type(G)

pandas.core.frame.DataFrame

In [17]:
G.iloc[diag_indices, diag_indices] += _lambda  # X'X + λI

In [18]:
P = np.linalg.inv(G)  # P = (X'X + λI)^(-1)

In [19]:
P.shape

(6807, 6807)

In [20]:
B = P / -np.diag(P)  # - P_{ij} / P_{jj} if i ≠ j

In [21]:
B

array([[-1.00000000e+00,  5.72312205e-03, -3.12144018e-03, ...,
         2.20171796e-04, -5.97970198e-04,  3.64565163e-04],
       [ 1.24224480e-02, -1.00000000e+00,  1.69766355e-03, ...,
         8.07563871e-04,  1.41682647e-03, -1.76039902e-03],
       [-2.72201023e-02,  6.82043670e-03, -1.00000000e+00, ...,
        -6.09152550e-04,  8.62072733e-04, -8.95349633e-04],
       ...,
       [ 3.74296144e-02,  6.32493940e-02, -1.18753111e-02, ...,
        -1.00000000e+00, -3.30790563e-02, -1.95220984e-03],
       [-3.63978582e-02,  3.97318487e-02,  6.01735259e-03, ...,
        -1.18439273e-02, -1.00000000e+00,  6.16586593e-03],
       [ 3.27141630e-02, -7.27775758e-02, -9.21337121e-03, ...,
        -1.03046555e-03,  9.08988598e-03, -1.00000000e+00]])

In [22]:
B.shape

(6807, 6807)

In [23]:
min_dim = min(B.shape)

In [24]:
min_dim

6807

In [25]:
B[range(min_dim), range(min_dim)] = 0  # 대각행렬 원소만 0으로 만들어주기 위해

In [26]:
B

array([[ 0.        ,  0.00572312, -0.00312144, ...,  0.00022017,
        -0.00059797,  0.00036457],
       [ 0.01242245,  0.        ,  0.00169766, ...,  0.00080756,
         0.00141683, -0.0017604 ],
       [-0.0272201 ,  0.00682044,  0.        , ..., -0.00060915,
         0.00086207, -0.00089535],
       ...,
       [ 0.03742961,  0.06324939, -0.01187531, ...,  0.        ,
        -0.03307906, -0.00195221],
       [-0.03639786,  0.03973185,  0.00601735, ..., -0.01184393,
         0.        ,  0.00616587],
       [ 0.03271416, -0.07277758, -0.00921337, ..., -0.00103047,
         0.00908989,  0.        ]])

In [27]:
X = X.to_numpy()

In [28]:
result = X[:,:] @ B

In [29]:
result

array([[ 1.25712227,  0.25051692, -0.05381435, ..., -0.01097048,
        -0.00971653,  0.02222529],
       [ 0.80351115, -0.02889054,  0.09607394, ...,  0.02164702,
        -0.00776703,  0.00874192],
       [-0.01070477, -0.11127   , -0.07181698, ..., -0.00550384,
        -0.0199763 ,  0.00305959],
       ...,
       [ 0.95999664,  0.39991291,  0.01827393, ...,  0.03032355,
        -0.03442212,  0.02122535],
       [ 0.21180574, -0.09025958,  0.00912311, ..., -0.01107915,
         0.01326027, -0.01430808],
       [ 1.25502847,  0.86698173,  0.01150279, ...,  0.02164161,
         0.01793866, -0.01068675]])

In [30]:
result[X.nonzero()] = -np.inf

In [31]:
result

array([[       -inf,  0.25051692, -0.05381435, ..., -0.01097048,
        -0.00971653,  0.02222529],
       [       -inf, -0.02889054,  0.09607394, ...,  0.02164702,
        -0.00776703,  0.00874192],
       [-0.01070477, -0.11127   , -0.07181698, ..., -0.00550384,
        -0.0199763 ,  0.00305959],
       ...,
       [       -inf,  0.39991291,  0.01827393, ...,  0.03032355,
        -0.03442212,  0.02122535],
       [ 0.21180574, -0.09025958,  0.00912311, ..., -0.01107915,
         0.01326027, -0.01430808],
       [       -inf,        -inf,  0.01150279, ...,  0.02164161,
         0.01793866, -0.01068675]])

In [32]:
result.shape

(31360, 6807)

In [33]:
len(result[0])

6807

In [34]:
tem = np.argsort(result[0])[-11:-1]

In [35]:
len(tem)

10

In [61]:
tem.tolist()

[2164, 2683, 1719, 2187, 4581, 2619, 41, 1666, 4463, 4597]

In [37]:
!pip install pandas==1.2.3
!pip install numpy==1.19.5

In [38]:
pd.__version__

'1.2.3'

In [39]:
np.__version__

'1.19.5'

In [40]:
submission = pd.DataFrame(columns=['user','item'])

In [41]:
# 유저수와 아이템 수 기록.
num_users = train['user'].nunique()
num_items = train['item'].nunique()

# user_id 모아둔 리스트 만들기.
user_lst = list(train['user'].unique())

In [57]:
submission_user_lst = []

for user in user_lst:
    for i in range(10):
        submission_user_lst.append(user)

In [59]:
submission['user'] = submission_user_lst

In [60]:
submission.head()

,user,item
0,11,NaN
1,11,NaN
2,11,NaN
3,11,NaN
4,11,NaN


In [62]:
submission_item_lst = []

for i in range(len(result)):
    tem = np.argsort(result[i])[-11:-1]
    submission_item_lst += tem.tolist()

In [63]:
submission_item_lst

[2164,
 2683,
 1719,
 2187,
 4581,
 2619,
 41,
 1666,
 4463,
 4597,
 4597,
 326,
 580,
 2653,
 1147,
 5202,
 1004,
 672,
 1335,
 1089,
 1489,
 657,
 5522,
 3433,
 4965,
 1766,
 179,
 2309,
 3057,
 1281,
 0,
 1016,
 360,
 1284,
 41,
 1666,
 3567,
 706,
 580,
 1532,
 1202,
 6173,
 6572,
 5218,
 6084,
 3013,
 3909,
 6410,
 5788,
 3200,
 4057,
 4640,
 2187,
 0,
 1532,
 4646,
 226,
 3038,
 2344,
 2653,
 76,
 1532,
 2624,
 659,
 646,
 4787,
 2659,
 2619,
 1580,
 647,
 1756,
 3068,
 3534,
 423,
 2304,
 416,
 136,
 2616,
 3200,
 317,
 750,
 1039,
 49,
 1249,
 1892,
 2138,
 848,
 885,
 424,
 693,
 4466,
 5936,
 2344,
 5652,
 653,
 4886,
 672,
 5461,
 2304,
 2659,
 580,
 5059,
 1612,
 1532,
 2852,
 358,
 5461,
 2344,
 1642,
 2664,
 5762,
 5788,
 3010,
 1580,
 3635,
 317,
 31,
 2309,
 413,
 1435,
 423,
 585,
 3654,
 717,
 209,
 644,
 1103,
 1612,
 1642,
 1016,
 644,
 353,
 585,
 786,
 619,
 784,
 649,
 646,
 3399,
 1095,
 644,
 4896,
 1887,
 5792,
 178,
 3041,
 3200,
 317,
 3654,
 593,
 471,
 527

In [65]:
submission['item'] = submission_item_lst

In [67]:
submission

,user,item
0,11,2164
1,11,2683
2,11,1719
3,11,2187
4,11,4581
...,...,...
313595,138493,2661
313596,138493,931
313597,138493,717
313598,138493,4107


In [68]:
submission.reset_index(drop=True, inplace=True)
submission.to_csv("../output/suyeon_EASE.csv", index=False)